# **Agricultural Mandi Price Announcements**

This notebook demonstrates how to build a multilingual crop price announcement system
for Indian farmers using Sarvam AI's language and speech models.

### **Use Case**

Fetch current crop prices from mandi (wholesale market) data and broadcast
natural-sounding announcements to farmers in their local language — Hindi, Tamil,
Telugu, Kannada, Marathi, or Gujarati.

### **Pipeline**

1. **Fetch:** Load synthetic mandi price data for the requested crop (price per quintal,
   market name, date).
2. **Announce:** Use **Sarvam-M** to format a natural-sounding public address announcement
   in the target language.
3. **Speak:** Use **Bulbul v3 TTS** to synthesize the announcement as a WAV audio file
   saved to `outputs/`.

### **Supported Crops**

wheat, rice, cotton, sugarcane, onion, potato, tomato

### **Supported Languages**

| Language | Code |
| :--- | :--- |
| Hindi | hi-IN |
| Tamil | ta-IN |
| Telugu | te-IN |
| Kannada | kn-IN |
| Marathi | mr-IN |
| Gujarati | gu-IN |

In [ ]:
# Pinning versions for reproducibility
!pip install -Uqq sarvamai>=0.1.24 python-dotenv>=1.0.0

### **1. Setup & API Key**

Obtain your API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai).
Create a `.env` file in this directory with `SARVAM_API_KEY=your_key_here`, or set the
environment variable directly before launching Jupyter.

In [ ]:
from __future__ import annotations

import base64
import datetime
import os
import traceback
from dataclasses import dataclass
from pathlib import Path

from dotenv import load_dotenv
from sarvamai import SarvamAI

load_dotenv()

SARVAM_API_KEY = os.environ.get("SARVAM_API_KEY", "")
if not SARVAM_API_KEY or SARVAM_API_KEY == "YOUR_SARVAM_API_KEY":
    raise RuntimeError(
        "SARVAM_API_KEY is not set. Add it to your .env file or set the environment variable."
    )

client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

print("Client initialised.")

### **2. Step 1 — FETCH: Load Mandi Price Data**

`get_crop_price` looks up the current price for a given crop from the synthetic mandi
dataset. In a production system this would call the Agmarknet API or a similar
government price feed.

Supported crops: wheat, rice, cotton, sugarcane, onion, potato, tomato

In [ ]:
@dataclass
class MandiRecord:
    """A single crop price record from a mandi."""

    crop: str
    price_per_quintal: float
    market: str
    date: str


# Synthetic demo date — replaced with today's date at runtime so the
# announcements always sound current when the notebook is run.
_TODAY = datetime.date.today().isoformat()

_MANDI_PRICES: dict[str, dict] = {
    "wheat":     {"price_per_quintal": 2275.0, "market": "Azadpur Mandi, Delhi",             "date": _TODAY},
    "rice":      {"price_per_quintal": 3500.0, "market": "Vashi Market, Mumbai",             "date": _TODAY},
    "cotton":    {"price_per_quintal": 6800.0, "market": "Rajkot Mandi, Gujarat",            "date": _TODAY},
    "sugarcane": {"price_per_quintal":  315.0, "market": "Pune Mandi, Maharashtra",          "date": _TODAY},
    "onion":     {"price_per_quintal": 1200.0, "market": "Lasalgaon Mandi, Nashik",          "date": _TODAY},
    "potato":    {"price_per_quintal": 1800.0, "market": "Agra Mandi, Uttar Pradesh",        "date": _TODAY},
    "tomato":    {"price_per_quintal": 2500.0, "market": "Kolar Mandi, Karnataka",           "date": _TODAY},
}

_SUPPORTED_CROPS = list(_MANDI_PRICES.keys())


def get_crop_price(crop: str) -> MandiRecord:
    """Return the mandi price record for the given crop.

    Args:
        crop: Crop name (case-insensitive). Supported: wheat, rice, cotton,
              sugarcane, onion, potato, tomato.

    Returns:
        MandiRecord with price, market, and date.

    Raises:
        ValueError: If the crop is not in the supported list.
    """
    key = crop.lower().strip()
    if key not in _MANDI_PRICES:
        raise ValueError(
            f"Unsupported crop: {crop!r}. "
            f"Supported crops: {', '.join(_SUPPORTED_CROPS)}"
        )
    data = _MANDI_PRICES[key]
    record = MandiRecord(
        crop=key,
        price_per_quintal=data["price_per_quintal"],
        market=data["market"],
        date=data["date"],
    )
    print(
        f"Fetched price — {record.crop}: "
        f"Rs {record.price_per_quintal:.2f}/quintal "
        f"at {record.market} on {record.date}"
    )
    return record


print("get_crop_price defined.")

### **3. Step 2 — ANNOUNCE: Format Price Announcement**

`format_announcement` sends the mandi record to **Sarvam-M** with a system prompt that
instructs the model to compose a short, natural-sounding public address announcement in
the target language.

The announcement includes the crop name, price per quintal, market name, and date —
phrased as a real mandi PA system would deliver it.

In [ ]:
_LANGUAGE_LABELS: dict[str, str] = {
    "hi-IN": "Hindi",
    "ta-IN": "Tamil",
    "te-IN": "Telugu",
    "kn-IN": "Kannada",
    "mr-IN": "Marathi",
    "gu-IN": "Gujarati",
}

_SUPPORTED_LANGUAGES = list(_LANGUAGE_LABELS.keys())

_ANNOUNCEMENT_SYSTEM_PROMPT = (
    "You are a mandi price announcer for an Indian agricultural wholesale market (mandi). "
    "Farmers and traders at the mandi rely on your announcements to make buying and "
    "selling decisions.\n\n"
    "Instructions:\n"
    "- Write the announcement entirely in the TARGET LANGUAGE specified in the request.\n"
    "- Keep it concise: 2-3 sentences.\n"
    "- Include the crop name, price per quintal in Indian rupees, market name, and date.\n"
    "- Use a clear, authoritative tone suitable for a public address system.\n"
    "- Do not add greetings, sign-offs, or explanatory notes outside the announcement text.\n"
    "- Write only the announcement text — nothing else."
)


def format_announcement(record: MandiRecord, language_code: str) -> str:
    """Format a natural-sounding mandi price announcement using Sarvam-M.

    Args:
        record:        MandiRecord containing crop, price, market, and date.
        language_code: BCP-47 target language code (e.g. 'hi-IN').

    Returns:
        Announcement text in the target language.

    Raises:
        ValueError: If the language code is unsupported or the model returns empty content.
    """
    if language_code not in _LANGUAGE_LABELS:
        raise ValueError(
            f"Unsupported language: {language_code!r}. "
            f"Supported codes: {', '.join(_SUPPORTED_LANGUAGES)}"
        )
    language_name = _LANGUAGE_LABELS[language_code]
    user_message = (
        f"TARGET LANGUAGE: {language_name} ({language_code})\n\n"
        f"Crop: {record.crop}\n"
        f"Price: Rs {record.price_per_quintal:.2f} per quintal\n"
        f"Market: {record.market}\n"
        f"Date: {record.date}"
    )

    response = client.chat.completions(
        messages=[
            {"role": "system", "content": _ANNOUNCEMENT_SYSTEM_PROMPT},
            {"role": "user",   "content": user_message},
        ],
        model="sarvam-m",
    )

    if not response or not response.choices:
        raise ValueError("Sarvam-M returned no response. Check your API quota.")

    content = response.choices[0].message.content
    if content is None:
        raise ValueError("Sarvam-M returned an empty message content.")

    announcement = content.strip()
    print(f"Announcement ({language_name}): {announcement}")
    return announcement


print("format_announcement defined.")

### **4. Step 3 — SPEAK: Synthesize Audio with Bulbul TTS**

`synthesize_announcement` converts the formatted announcement text to speech using
**Bulbul v3 TTS** and saves the WAV file to the `outputs/` folder.

Each language is paired with a natural-sounding speaker voice. The filename encodes
both the crop name and language code for easy identification.

In [ ]:
_SPEAKER_MAP: dict[str, str] = {
    "hi-IN": "shubh",
    "ta-IN": "kavya",
    "te-IN": "priya",
    "kn-IN": "arvind",
    "mr-IN": "shubh",
    "gu-IN": "priya",
}


def synthesize_announcement(
    text: str,
    crop: str,
    language_code: str,
    output_dir: str = "outputs",
) -> str:
    """Synthesize a mandi price announcement to WAV using Bulbul v3 TTS.

    Args:
        text:          Announcement text to synthesize.
        crop:          Crop name, used to build the output filename.
        language_code: BCP-47 language code (e.g. 'hi-IN').
        output_dir:    Directory where the WAV file is saved.

    Returns:
        Path to the saved WAV file.

    Raises:
        RuntimeError: If Bulbul TTS returns no audio data.
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    speaker = _SPEAKER_MAP.get(language_code, "shubh")

    tts_response = client.text_to_speech.convert(
        text=text,
        target_language_code=language_code,
        model="bulbul:v3",
        speaker=speaker,
        speech_sample_rate=24000,
        enable_preprocessing=True,
    )

    if not tts_response.audios:
        raise RuntimeError(
            f"Bulbul TTS returned no audio for language {language_code}. "
            "Check that the language code and speaker are supported."
        )

    audio_bytes = base64.b64decode(tts_response.audios[0])
    output_path = Path(output_dir) / f"mandi_{crop}_{language_code}.wav"
    output_path.write_bytes(audio_bytes)

    print(f"Audio saved to: {output_path}")
    return str(output_path)


print("synthesize_announcement defined.")

### **5. End-to-End Pipeline**

`announce_crop_price` ties all three steps together. Pass any supported crop name and
language code to receive a dict with the mandi record, formatted announcement text, and
path to the synthesized WAV file.

In [ ]:
def announce_crop_price(
    crop: str,
    language_code: str = "hi-IN",
    output_dir: str = "outputs",
) -> dict | None:
    """Full pipeline: fetch price -> format announcement -> synthesize audio.

    Args:
        crop:          Crop name (e.g. 'wheat', 'onion'). Case-insensitive.
        language_code: BCP-47 target language code (default: 'hi-IN').
        output_dir:    Directory where the WAV file is saved.

    Returns:
        Dict with keys 'crop', 'language_code', 'price_per_quintal', 'market',
        'date', 'announcement', and 'audio_path'; or None if the pipeline fails.
    """
    print(f"Processing: {crop} in {_LANGUAGE_LABELS.get(language_code, language_code)}")
    try:
        print("  Step 1/3 — Fetching mandi price data...")
        record = get_crop_price(crop)

        print("  Step 2/3 — Formatting announcement with Sarvam-M...")
        announcement = format_announcement(record, language_code)

        print("  Step 3/3 — Synthesizing audio with Bulbul TTS...")
        audio_path = synthesize_announcement(
            announcement, record.crop, language_code, output_dir
        )

        return {
            "crop":              record.crop,
            "language_code":     language_code,
            "price_per_quintal": record.price_per_quintal,
            "market":            record.market,
            "date":              record.date,
            "announcement":      announcement,
            "audio_path":        audio_path,
        }

    except Exception as e:
        traceback.print_exc()
        print(f"ERROR: Failed to process {crop!r}: {e}")
        return None


print("announce_crop_price defined.")

### **6. Demo — Generate Announcements for 5 Crops in Hindi**

The cell below runs the full pipeline for five crops — wheat, rice, onion, potato, and
tomato — with Hindi (`hi-IN`) as the target language.

No external data files are required; all mandi prices are drawn from the built-in
synthetic dataset.

In [ ]:
# Generate price announcements for 5 crops in Hindi
_demo_crops = ["wheat", "rice", "onion", "potato", "tomato"]
_demo_language = "hi-IN"

results = []
for crop in _demo_crops:
    print("-" * 60)
    result = announce_crop_price(crop, language_code=_demo_language)
    if result:
        results.append(result)

print("-" * 60)
print(f"Done. Generated {len(results)}/{len(_demo_crops)} announcements.")

### **7. Results**

Inspect the announcement text and listen to or download each WAV file below.

In [ ]:
from IPython.display import Audio, FileLink, display

if results:
    for i, result in enumerate(results, start=1):
        lang_label = _LANGUAGE_LABELS.get(result["language_code"], result["language_code"])
        print(f"--- Result {i}/{len(results)} ---")
        print(f"Crop          : {result['crop']}")
        print(f"Language      : {lang_label} ({result['language_code']})")
        print(f"Price         : Rs {result['price_per_quintal']:.2f} / quintal")
        print(f"Market        : {result['market']}")
        print(f"Announcement  : {result['announcement']}")
        print()
        display(Audio(filename=result["audio_path"]))
        display(FileLink(result["audio_path"], result_html_prefix="Download: "))
        print()
else:
    print("No results to display. Check the error messages above.")

### **8. Error Reference**

| Error | HTTP Status | Cause | Solution |
| :--- | :--- | :--- | :--- |
| `RuntimeError: SARVAM_API_KEY is not set` | — | Missing API key | Add key to `.env` |
| `invalid_api_key_error` | 403 | Invalid API key | Verify at [dashboard.sarvam.ai](https://dashboard.sarvam.ai) |
| `insufficient_quota_error` | 429 | Quota exceeded | Check usage limits |
| `internal_server_error` | 500 | Transient server issue | Wait and retry |
| `ValueError: Unsupported crop` | — | Crop not in dataset | Use a supported crop name |
| `ValueError: Unsupported language` | — | Language code not mapped | Use a supported BCP-47 code |
| `RuntimeError: no audio returned` | — | TTS speaker/language mismatch | Check `_SPEAKER_MAP` |

### **9. Using Additional Languages or Crops**

To announce a price in a different language, pass any supported language code:

```python
result = announce_crop_price("cotton",    language_code="ta-IN")  # Tamil
result = announce_crop_price("sugarcane", language_code="te-IN")  # Telugu
result = announce_crop_price("onion",     language_code="kn-IN")  # Kannada
result = announce_crop_price("potato",    language_code="mr-IN")  # Marathi
result = announce_crop_price("tomato",    language_code="gu-IN")  # Gujarati
```

To extend the dataset, add an entry to `_MANDI_PRICES` in the Fetch cell:

```python
_MANDI_PRICES["soybean"] = {
    "price_per_quintal": 4200.0,
    "market": "Indore Mandi, Madhya Pradesh",
    "date": "2024-01-15",
}
```

### **10. Conclusion & Resources**

This recipe chains **Sarvam-M** and **Bulbul v3 TTS** to produce ready-to-broadcast
agricultural price announcements in six Indian languages — no audio recording or
translation step required.

* [Sarvam AI Docs](https://docs.sarvam.ai)
* [Sarvam-M Chat API](https://docs.sarvam.ai/api-reference-docs/chat)
* [Bulbul TTS API](https://docs.sarvam.ai/api-reference-docs/text-to-speech)
* [Indic Language Support](https://docs.sarvam.ai/language-support)

**Keep Building!**